In [1]:
import os
from dotenv import load_dotenv
load_dotenv() # load_env
from pymongo import MongoClient
import pandas as pd
import time

cc = os.getenv("credit_card")
token = os.getenv("token")
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["Ironhack"]
c = db.get_collection("companies")


In [20]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [35]:
filter_1 = {
    "offices": {"$elemMatch": {"city": "Tokyo"}},
    "milestones": {"$elemMatch": {"description": "gaming"}}
}

projection = {"name": 1, "country_code":1, "_id":0}

query = list(c.find(filter_1, projection).sort("name", -1))
query

[]

In [62]:
filter_1 = {
    "tag_list": {"$regex": "gaming", "$options": "i"}
}

projection = {"_id":0, "offices.city":1}

query = list(c.find(filter_1, projection).sort("name", -1))
query

from collections import defaultdict

city_counts = defaultdict(int)

for item in query:
    for office in item['offices']:
        city = office.get('city', '')
        if city:  # Check if city is not empty or None
            city_counts[city] += 1

# Sort by value in descending order
sorted_city_counts = {k: v for k, v in sorted(city_counts.items(), key=lambda item: item[1], reverse=True)}

print(sorted_city_counts)


{'San Francisco': 14, 'New York': 13, 'London': 7, 'Los Angeles': 6, 'Paris': 5, 'Santa Monica': 4, 'Shanghai': 4, 'Redwood City': 4, 'Madrid': 2, 'Atlanta': 2, 'Helsinki': 2, 'Beverly Hills': 2, 'Austin': 2, 'Singapore': 2, 'Stockholm': 2, 'Therwil': 2, 'BogotÃ¡ D.C.': 1, 'Wilton': 1, 'Walnut Creek': 1, 'Munich': 1, 'Dundee': 1, 'Mellieha': 1, 'Cambridge': 1, 'Villamayor': 1, 'Westwood': 1, 'Amsterdam': 1, 'Wayne': 1, 'Buenos Aires': 1, 'Durham': 1, 'Agoura Hills': 1, 'Boca Raton': 1, 'Kolkata': 1, 'Berlin': 1, 'Dublin 8': 1, 'TromsÃ¸': 1, 'Mountain View': 1, 'Bangalore': 1, 'Woodbridge': 1, 'Vero Beach': 1, 'Nanterre cedex': 1, 'NÃ®mes cedex': 1, 'Brussels': 1, 'Frankfurt': 1, 'Milano': 1, 'Tokyo': 1, 'Baar ZG': 1, 'Lisboa': 1, 'San Fransisco': 1, 'SÃ¢o Paulo': 1, 'Sartell': 1, 'Minneapolis': 1, 'Palo Alto': 1, 'Hilton': 1, 'Delhi': 1, 'Sunnyvale': 1, 'Alderney': 1, 'Hailey': 1, 'West Hollywood': 1, 'Cologne': 1, 'Chicago': 1, 'Navi-Mumbai': 1, 'Dhaka': 1, 'Worcester': 1, 'Taipei': 1

In [53]:
import pandas as pd

# Take top 10 cities
top_10_cities = dict(list(sorted_city_counts.items())[:10])

# Create DataFrame
df = pd.DataFrame(list(top_10_cities.items()), columns=['City', 'Count'])

df


,City,Count
0,San Francisco,14
1,New York,13
2,London,7
3,Los Angeles,6
4,Paris,5
5,Santa Monica,4
6,Shanghai,4
7,Redwood City,4
8,Madrid,2
9,Atlanta,2


In [58]:
def find_top_10_gaming_cities(c):
    filter_1 = {
        "tag_list": {"$regex": "gaming", "$options": "i"}
    }

    projection = {"_id": 0, "offices.city": 1}

    query = list(c.find(filter_1, projection).sort("name", -1))

    city_counts = defaultdict(int)

    for item in query:
        for office in item['offices']:
            city = office.get('city', '')
            if city:  # Check if city is not empty or None
                city_counts[city] += 1

    # Sort by value in descending order
    sorted_city_counts = {k: v for k, v in sorted(city_counts.items(), key=lambda item: item[1], reverse=True)}
    
    # Take top 10 cities
    top_10_cities = dict(list(sorted_city_counts.items())[:10])

    return pd.DataFrame(list(top_10_cities.items()), columns=['City', 'Count'])


#### Based on the numerous amount of gaming companies that are as well in San Francisco, I will be looking as well into location my company there. 

In [59]:
df = find_top_10_gaming_cities(c)
df

,City,Count
0,San Francisco,14
1,New York,13
2,London,7
3,Los Angeles,6
4,Paris,5
5,Santa Monica,4
6,Shanghai,4
7,Redwood City,4
8,Madrid,2
9,Atlanta,2
